In [8]:
import json
import networkx as nx
import torch

device = 'cpu'

In [9]:
def dict2tensor(d, fill=None):
    n = len(d.keys())
    k2i = {k:i for i,k in enumerate(sorted(d.keys()))}
    res = torch.zeros(len(d.keys()), len(d.keys()), device=device)
    for src_node, dst_nodes in d.items():
        for dst_node, distance in dst_nodes.items():
            if fill is not None:
                res[k2i[src_node],k2i[dst_node]] = fill
            else:
                res[k2i[src_node],k2i[dst_node]] = distance
    return res, k2i

In [11]:
%%time

print('generating graph...')
# G = nx.path_graph(10)
# G = nx.cycle_graph(10)
# G = nx.balanced_tree(2,3)
# G = nx.connected_watts_strogatz_graph(10,5,0.5)
# G = file2graph('./facebook/0.edges')
# G = file2graph('./facebook/107.edges')
# G = file2graph('./facebook/414.edges')
initPositions = None

##cube
G = nx.Graph()
G.add_nodes_from(range(8))
G.add_edges_from([
    (0,1),(1,2),(2,3),(3,0),
    (4,5),(5,6),(6,7),(7,4),
    (0,4),(1,5),(2,6),(3,7)])
initPositions = None

# # cross
# G = nx.Graph()
# G.add_nodes_from([0,1,2,3])
# G.add_edges_from([(0,1),(2,3)])
# initPositions = [[0,0],[7,1],[7,0],[0,1]]

#dodecahedron
# https://www.researchgate.net/figure/The-dodecahedron-graph_fig5_275536736
# G = nx.Graph()
# G.add_nodes_from(range(1,21,1))
# G.add_edges_from([
#     (1,2),(2,3),(3,4),(4,5),(5,1),
#     (6,7),(7,8),(8,11),(11,13),(13,14),(14,15),(15,17),(17,16),(16,18),(18,6),
#     (9,10),(10,12),(12,20),(20,19),(19,9),
#     (1,16),(5,6),(4,8),(3,13),(2,15),
#     (12,14),(17,20),(18,19),(7,9),(10,11),
# ])
# initPositions = None


# K5
G = nx.complete_graph(5)


print(len(G.nodes), 'nodes')
print('calculating all pairs shortest path...')
D, k2i = dict2tensor(dict(nx.all_pairs_shortest_path_length(G)))
# D += torch.eye(D.shape[0])
W = 1/(D**2+1)

Adj,_ = dict2tensor(dict(G.adjacency()), fill=1)
print('\n\n')

neighborSizes = Adj.sum(dim=1).int()
nodeCount = Adj.shape[0]
eye = torch.eye(Adj.shape[0], device=device)
truth = Adj + eye


generating graph...
5 nodes
calculating all pairs shortest path...



CPU times: user 2.17 ms, sys: 0 ns, total: 2.17 ms
Wall time: 1.33 ms


In [12]:
def graph2json(G):
    return {
        'nodes': [
            {
                'index':i, 
                'id': n
            } 
            for i, n in enumerate(G.nodes)
        ],
        'edges': [
            {
                'source': e1, 
                'target': e2
            } 
            for e1,e2 in G.edges
        ],
        'weight': W.numpy().tolist(),
        'graphDistance': D.numpy().tolist(),
        'initPositions': initPositions
    }

with open('graph.json', 'w') as f:
    json.dump(graph2json(G), f, indent=2)